In [ ]:
import numpy as np
import wandb
from matplotlib import pyplot as plt
from util import populate_plt_settings, get_column_width, get_fig_size, get_latex_float
import re
import plotly.express as px
import pandas as pd

In [ ]:
populate_plt_settings(plt)

In [ ]:
api = wandb.Api()
run = api.run('kennychufk/alluvion-rl/fa8dq4m1Aug-diagonal100')

In [ ]:
history = run.scan_history(keys=None,
                           page_size=1000,
                           min_step=None,
                           max_step=None)

In [ ]:
metric_name = 'eulerian_masked'
# metric_name = 'eulerian'
pattern=re.compile('([0-9]+)-uth-.*-'+metric_name+'-m%')

buoy_counts_in_trials = []

step_interval = 50
for row in history:
    if len(buoy_counts_in_trials)>0:
        continue
    for key in row:
        match_result = pattern.match(key)
        if match_result is not None:
            num_buoys = int(match_result[1])
            buoy_counts_in_trials.append(num_buoys)
buoy_counts_in_trials.sort()

buoy_counts_in_trials
buoy_counts_grouped = []
curr_buoy_count = -1
curr_idx =-1
for buoy_count in buoy_counts_in_trials:
    if curr_buoy_count!=buoy_count:
        buoy_counts_grouped.append([])
        curr_buoy_count = buoy_count
        curr_idx+=1
    buoy_counts_grouped[curr_idx].append(buoy_count)
    
buoy_count_to_id = {}
for idx, buoy_counts_same in enumerate(buoy_counts_grouped):
    buoy_count_to_id[buoy_counts_same[0]] = idx
    

val_scores = []
for buoy_counts_same in buoy_counts_grouped:
    val_scores.append(np.zeros(len(buoy_counts_same)))

artifact_id = 0
cursors = np.zeros(len(buoy_counts_grouped), int)
for row_id, row in enumerate(history):
    if (row['_step']+1)%50!=0:
        continue
    for key in row:
        match_result = pattern.match(key)
        if match_result is not None:
            num_buoys = int(match_result[1])
            buoy_count_index = buoy_count_to_id[num_buoys]
            val_scores[buoy_count_index][cursors[buoy_count_index]] = 1-row[key]
            cursors[buoy_count_index]+=1
    artifact_id+=1
distinct_buoy_counts = []
for buoy_counts_same in buoy_counts_grouped:
    distinct_buoy_counts.append(buoy_counts_same[0])

In [ ]:
buoy_counts_grouped

In [ ]:
for i in range(len(buoy_counts_grouped)):
    plt.scatter(buoy_counts_grouped[i], val_scores[i])

In [ ]:
score_std = np.zeros(len(val_scores))
score_mean = np.zeros(len(val_scores))

for i, scores in enumerate(val_scores):
    score_std[i] = scores.std()
    score_mean[i] = scores.mean()

In [ ]:
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
num_rows = 1
num_cols = 1
fig, ax = plt.subplots(num_rows, num_cols, figsize = get_fig_size(get_column_width(), ratio=(np.sqrt(5)-1)*0.5))

# ax.plot(distinct_buoy_counts, score_mean)
# ax.fill_between(distinct_buoy_counts, score_mean - score_std, score_mean + score_std, color='b', alpha=0.2)

ax.errorbar(distinct_buoy_counts, score_mean, yerr=score_std, capsize=2, capthick=1)
ax.set_xlabel('Number of buoys')
ax.set_ylabel('Eulerian Score')

ax.set_xlim(0,100)
ax.xaxis.set_major_locator(MultipleLocator(10))
ax.xaxis.set_minor_locator(AutoMinorLocator(5))
ax.grid(which='minor', color='#DDDDDD', linewidth=0.5)

fig.tight_layout(pad=0.05) # should set tight_layout before add_axes()
fig.savefig('buoy-count-score-correlation.pgf')

In [ ]:
score_std